In [1]:
from datetime import datetime
import time
import requests
import pickle
from pathlib import Path
import re

In [2]:
def print_log(*args):
    print(f"[{str(datetime.now())[:-3]}] ", end="")
    print(*args)

In [3]:
def get_search_results(params):
    req_sr = requests.get(
        "https://store.steampowered.com/search/results/",
        params=params)
    
    if req_sr.status_code != 200:
        print_log(f"Failed to get search results: {req_sr.status_code}")
        return {"items": []}
    
    try:
        search_results = req_sr.json()
    except Exception as e:
        print_log(f"Failed to parse search results: {e}")
        return {"items": []}
    
    return search_results

In [4]:
def get_app_details(appid):
    while(True):
        if appid == None:
            print_log("App Id is None.")
            return {}

        appdetails_req = requests.get(
            "https://store.steampowered.com/api/appdetails/",
            params={"appids": appid, "cc": "hk", "l": "english"})
        
        if appdetails_req.status_code == 200:
            appdetails = appdetails_req.json()
            appdetails = appdetails[str(appid)]
            print_log(f"App Id: {appid} - {appdetails['success']}")
            break

        elif appdetails_req.status_code == 429:
            print_log(f'Too many requests. Sleep for 10 sec')
            time.sleep(10)
            continue

        elif appdetails_req.status_code == 403:
            print_log(f'Forbidden to access. Sleep for 5 min.')
            time.sleep(5 * 60)
            continue

        else:
            print_log("ERROR: status code:", appdetails_req.status_code)
            print_log(f"Error in App Id: {appid}.")
            appdetails = {}
            break

    return appdetails

In [5]:
execute_datetime = datetime.now()

search_result_folder_path = Path(f"search_results_{execute_datetime.strftime('%Y%m%d')}")
if not search_result_folder_path.exists():
    search_result_folder_path.mkdir()

In [6]:
# a list of filters
params_list = [
    {"filter": "topsellers"},
    {"filter": "globaltopsellers"},
    {"filter": "popularnew"},
    {"filter": "popularcommingsoon"},
    {"filter": "", "specials": 1}
]
page_list = list(range(1, 5))

params_sr_default = {
    "filter": "topsellers",
    "hidef2p": 1,
    "page": 1,              # page is used to go through different parts of the ranking. Each page contains 25 results
    "json": 1
}

for update_param in params_list:

    items_all = []
    if update_param["filter"]:
        filename = f"{update_param['filter']}_{execute_datetime.strftime('%Y%m%d')}.pkl"
    else:
        filename = f"specials_{execute_datetime.strftime('%Y%m%d')}.pkl"

    if (search_result_folder_path / filename).exists():
        print_log(f"File {filename} exists. Skip.")
        continue

    for page_no in page_list:
        param = params_sr_default.copy()
        param.update(update_param)
        param["page"] = page_no

        search_results = get_search_results(param)
        print_log(search_results)

        if not search_results:
            continue

        items = search_results.get("items", [])

        # proprocessing search results to retrieve the appid of the game
        for item in items:
            try:
                item["appid"] = re.search(r"steam/\w+/(\d+)", item["logo"]).group(1)      # the URL can be steam/bundles/{appid} or steam/apps/{appid}
            except Exception as e:
                print_log(f"Failed to extract appid: {e}")
                item["appid"] = None

        # request for game information using appid
        for item in items:
            appid = item["appid"]
            appdetails = get_app_details(appid)
            item["appdetail"] = appdetails

        items_all.extend(items)

    # save the search results
    with open(search_result_folder_path / filename, "wb") as f:
        pickle.dump(items_all, f)
    print_log(f"Saved {filename}")

[2024-11-17 17:18:26.751] {'desc': '', 'items': [{'name': 'Call of Duty®: Black Ops 6', 'logo': 'https://shared.fastly.steamstatic.com/store_item_assets/steam/apps/2933620/capsule_sm_120.jpg?t=1731604753'}, {'name': 'S.T.A.L.K.E.R. 2: Heart of Chornobyl', 'logo': 'https://shared.fastly.steamstatic.com/store_item_assets/steam/apps/1643320/ffcddbc0c191d569b783cd8534c64b3968abc2db/capsule_sm_120.jpg?t=1731627913'}, {'name': 'Steam Deck', 'logo': 'https://shared.fastly.steamstatic.com/store_item_assets/steam/apps/1675200/capsule_sm_120.jpg?t=1699990406'}, {'name': 'Farming Simulator 25', 'logo': 'https://shared.fastly.steamstatic.com/store_item_assets/steam/apps/2300320/capsule_sm_120.jpg?t=1731674498'}, {'name': 'Hearts of Iron IV', 'logo': 'https://shared.fastly.steamstatic.com/store_item_assets/steam/apps/394360/d63499ddaf322a6f7bcf9adf2e7d5876d5489beb/capsule_sm_120_alt_assets_2.jpg?t=1731604776'}, {'name': 'Enshrouded', 'logo': 'https://shared.fastly.steamstatic.com/store_item_assets/

In [7]:
date_to_read = datetime(2024, 11, 17)

items_listlist = []

for file in search_result_folder_path.iterdir():
    if file.suffix == ".pkl":
        date = datetime.strptime(file.stem.split("_")[-1], "%Y%m%d")
        if date == date_to_read:
            with open(file, "rb") as f:
                items = pickle.load(f)
            print_log(f"Read {file}")
            items_listlist.append(items)
            

[2024-11-17 17:30:45.899] Read search_results_20241117\globaltopsellers_20241117.pkl
[2024-11-17 17:30:46.015] Read search_results_20241117\popularcommingsoon_20241117.pkl
[2024-11-17 17:30:47.236] Read search_results_20241117\popularnew_20241117.pkl
[2024-11-17 17:30:47.325] Read search_results_20241117\specials_20241117.pkl
[2024-11-17 17:30:47.457] Read search_results_20241117\topsellers_20241117.pkl
